In [30]:
SOURCE_MARKDOWN = """
**Subject: The Wordpress vs WP Engine Drama**

In a surprising twist of events, the open-source giant WordPress is entangled in a dramatic controversy surrounding its licensing policies aimed squarely at WP Engine, stirring fierce discussions across the web. **"How can you expect any goodwill towards open-source from a community when you insist on enforcing fees?"** This quote captures the mounting frustration many in the tech community feel as these events unfold.

**Background Context**  
WordPress, an open-source content management system, has thrived on its community-driven model for years. However, recently, the organization faced backlash due to new licensing fee structures selectively impacting WP Engine, a leading managed Wordpress host. This move has raised concerns among other hosting companies and developers about potential instability and implications for open-source projects.

---

**The Emergence of Tension**  
The friction began to surface as Automattic, the parent company of WordPress, targeted WP Engine for allegedly not contributing enough to the WordPress ecosystem. **James Ivings** pointed out in one of his tweets, *“You can’t go IPO with just a happy customer base, you need to be extracting profits from the entire market (via licensing).”* This sentiment echoes the belief that the motives behind the licensing changes extend beyond simply supporting the open-source initiative. ([source](https://twitter.com/jamesivings/status/1839422193681481750))

---

**Mixed Reactions Among Developers**  
As the news broke, reaction from the community was polarized. While some developers supported Automattic's actions as a means to ensure sustainable growth, others voiced concerns about stifling innovation and creating a hostile environment for open source. One developer remarked that experienced consequences, like **“Being blocked from installing plugins,”** could negatively impact WP Engine's customer base and, subsequently, WordPress's reputation as a stable and reliable platform. ([source](https://twitter.com/arvidkahl/status/1839445536686174387))

---

**The Financial Perspective**  
Conversations centered around revenue models emerged, as seen in **Danny Postmaa's** reflections on his experiences, *"more growth != more support tickets."* He noted that despite their expanding user base, support requests remained steady, indicating a deeper complexity in managing resources amid growth. ([source](https://twitter.com/dannypostmaa/status/1839847665338925293))

There’s a growing belief that this drama could lead to significant changes in how hosting services operate with WordPress, resulting in a shift towards licensing strategies that others, like NewFoldDigital, have already embraced. Many felt securing licensing is a smart strategic move, potentially raising the licensing fees for all WP hosting partners. ([source](https://twitter.com/jessethanley/status/1839569215000588641))

---

**Community Support and Reactions**  
While the turmoil has pushed various developers to share their thoughts online, there’s a palpable sense of disbelief among users and developers alike. The unforeseen changes raise questions about open-source integrity, As **Arvid Kahl** noted, emphasizing the need for transparency from Automattic in their reasoning. *“I hope the ecosystem is self-healing. I just hope WPE being blocked from installing can be healed without causing massive reputational damage,”* he expressed. ([source](https://twitter.com/arvidkahl/status/1839445536686174387))

---

In conclusion, the **WordPress vs. WP Engine drama** reveals not just a licensing issue but digs into the larger questions surrounding the sustainability of open-source models and the balance of growth, community trust, and corporate influence. As this saga unfolds, the tech community watches closely, ready to respond.
"""


In [31]:
from typing import Literal, Optional, List, Union
from pydantic import BaseModel, Field

STORYBOARD_PROMPT = """
I will provide you with a source string that represents a news article.

Your job is to create a storyboard for a video that will be generated from the news article.

The storyboard should be an array of objects, where each object represents a starting frame for a video, which will then be animated to create a video segment with a fixed duration.

Don't put two of the same type in a row, and use at least one of each type.

I will also supply a total duration for the video, generate enough frames to fill the duration given a duration of 2 seconds for each frame. Print out the total number of frames you will generate.
"""


TYPE_DESCRIPTION = {
    "meme": "a meme image, that will be animated",
    "twitter_screenshot": "a screenshot of a twitter thread",
    "stock_video": "a stock video based on an initial image",
}


STOCK_IMAGE_DESCRIPTION = """
A description of an image that will be used as a prompt to generate a stock image. We want the image to be photo realistic, showing a scene that matches the source markdown in some way.
"""


class StoryboardItem(BaseModel):
    type: Literal["meme", "twitter_screenshot", "stock_video"] = Field(
        ..., description=str(TYPE_DESCRIPTION)
    )
    stock_image_description: Union[str, None] = Field(
        ..., description=STOCK_IMAGE_DESCRIPTION
    )
    twitter_url: Union[str, None] = Field(
        ...,
        description="The url of the twitter thread to screenshot if the type is twitter_screenshot. Leave blank if the type is not twitter_screenshot. Make sure the tweet is in the source markdown.",
    )


class Storyboard(BaseModel):
    items: List[StoryboardItem] = Field(..., description="The storyboard items")
    total_duration: int = Field(
        ..., description="The total duration of the video in seconds"
    )
    total_frames: int = Field(
        ..., description="The total number of frames in the video"
    )


In [32]:
import os
from openai import OpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI()

TOTAL_DURATION = 60

# Call OpenAI API with GPT-4 Turbo
response = client.beta.chat.completions.parse(
    model="gpt-4o-2024-08-06",
    messages=[
        {"role": "system", "content": STORYBOARD_PROMPT},
        {"role": "user", "content": SOURCE_MARKDOWN},
        {
            "role": "user",
            "content": f"The total duration of the video is {TOTAL_DURATION} seconds.",
        },
    ],
    response_format=Storyboard,
)

# Extract the generated storyboard
storyboard = response.choices[0].message.parsed


print("Storyboard: ")
print("Duration: ", storyboard.total_duration)
print("Total Frames: ", storyboard.total_frames)
for item in storyboard.items:
    print(item)


Storyboard: 
Duration:  60
Total Frames:  30
type='meme' stock_image_description='A digital illustration of characters representing WordPress and WP Engine in a courtroom setting, reflecting drama and controversy.' twitter_url=None
type='twitter_screenshot' stock_image_description=None twitter_url='https://twitter.com/jamesivings/status/1839422193681481750'
type='stock_video' stock_image_description='A stock video of a gavel hitting a desk, symbolizing legal decisions and controversies.' twitter_url=None
type='meme' stock_image_description='A graphic showing scales of justice, with open-source ideals on one side and licensing fees on the other.' twitter_url=None
type='twitter_screenshot' stock_image_description=None twitter_url='https://twitter.com/arvidkahl/status/1839445536686174387'
type='stock_video' stock_image_description='A video clip of a bustling tech office, representing the developer community in discussion.' twitter_url=None
type='meme' stock_image_description='A meme with 

In [23]:
%reload_ext autoreload
%autoreload 2


import requests

IDEOGRAM_URL = "https://api.ideogram.ai/generate"

IDEOGRAM_HEADERS = {
    "Api-Key": os.getenv("IDEOGRAM_API_KEY"),
    "Content-Type": "application/json",
}


def generate_ideo_image(prompt: str, starting_image_url: Optional[str] = None):
    image_request = {
        "image_request": {
            "prompt": prompt,
            "model": "V_2",
            "magic_prompt_option": "AUTO",
        }
    }

    if starting_image_url:
        image_request["image_request"]["keyframe"] = {
            "frame0": {"type": "image", "url": starting_image_url}
        }

    image_response = requests.post(
        IDEOGRAM_URL, json=image_request, headers=IDEOGRAM_HEADERS
    )
    return image_response.json()


In [33]:
from tweetcapture import TweetCapture
import asyncio


import os

async def capture_tweet(url, port):
    try:
        tweet = TweetCapture()
        tweet.add_chrome_argument(f"--remote-debugging-port={port}")
        
        # Create 'tweets' directory if it doesn't exist
        os.makedirs('tweets', exist_ok=True)
        
        # Generate a filename based on the URL and username
        username = url.split('/')[-3]  # Assuming the URL format is twitter.com/username/status/id
        filename = f"tweets/{username}_{url.split('/')[-1]}.png"
        
        await tweet.screenshot(url, path=filename, overwrite=True)
        return filename
    except Exception as e:
        print(f"Error capturing tweet {url}: {str(e)}")
        return None


async def capture_tweets(tweet_urls):
    port = 9222
    tasks = []
    for url in tweet_urls:
        tasks.append(asyncio.create_task(capture_tweet(url, port)))
        port += 1

    filenames = []
    for task in asyncio.as_completed(tasks):
        try:
            filename = await task
            if filename:
                filenames.append(filename)
        except Exception as e:
            print(f"Error processing task: {str(e)}")

    return filenames







In [34]:
from lumaai import LumaAI
import time

MAX_ATTEMPTS = 30
POLL_INTERVAL = 5

client = LumaAI()

def generate_luma_video(prompt: Optional[str] = None, start_image_url: Optional[str] = None, aspect_ratio: str = "16:9" ):
    generation = client.generations.create(
        prompt=prompt,
        keyframes={
            "frame0": {
                "type": "image",
                "url": start_image_url
            }
        } if start_image_url else {},
        aspect_ratio=aspect_ratio
    )
    return generation.id


def poll_generation(generation_id, max_attempts=MAX_ATTEMPTS, delay=POLL_INTERVAL):
    start_time = time.time()
    for attempt in range(max_attempts):
        status = client.generations.get(generation_id)
        if status.state == "completed":
            end_time = time.time()
            return status, end_time - start_time
        elif status.state == "failed":
            raise Exception(f"Generation failed: {status.failure_reason}")

In [37]:

import asyncio

async def process_twitter_screenshot(item):
    if not item.twitter_url:
        raise ValueError("Twitter URL is required for twitter_screenshot")
    filenames = await capture_tweets([item.twitter_url])
    return {
        "type": "twitter_screenshot",
        "url": filenames[0] if filenames else None
    }

async def process_stock_video(item):
    ideogram_response = generate_ideo_image(item.stock_image_description)   
    # luma_video_id = generate_luma_video(item.stock_image_description)
    print(ideogram_response)
    return {
        "type": "stock_video",
        "url": ideogram_response["data"][0]["url"]
    }

async def process_item(item):
    print("Processing item: ", item.type)
    if item.type == "twitter_screenshot":
        return await process_twitter_screenshot(item)
    elif item.type == "stock_video":
        return await process_stock_video(item)

async def process_all_items():
    tasks = [process_item(item) for item in storyboard.items]
    outputs = await asyncio.gather(*tasks)
    return outputs

outputs = await process_all_items()
print(outputs)



{'created': '2024-09-28T20:24:14.478428+00:00', 'data': [{'is_image_safe': True, 'prompt': 'A stock video of a gavel hitting a wooden desk. The gavel is a symbol of authority and legal decisions. The wooden desk has a green cloth draped over it. A chair is placed behind the desk. The background is a room with beige walls and a window with curtains.', 'resolution': '1024x1024', 'seed': 697038616, 'style_type': 'GENERAL', 'url': 'https://ideogram.ai/api/images/ephemeral/Cfz1-tPoQuKcpqMumW88GA.png?exp=1727641474&sig=188ac48b917eb34ce655accdb347839cec68ebd71104e4b03a0f619752aa496e'}]}
{'created': '2024-09-28T20:24:35.235087+00:00', 'data': [{'is_image_safe': True, 'prompt': 'A video clip of a bustling tech office with a diverse developer community in discussion. There are multiple workstations with computers, some with multiple monitors. Developers are seated at the workstations, engaged in conversation. There are whiteboards with diagrams in the background.', 'resolution': '1024x1024', 's